In [2]:
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

In [3]:
df_raw = pd.read_excel('../data/data.xlsx')

In [4]:
df_raw['Dual']

0                                                    NaN
1                                                    NaN
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
546               13 MP, f/2.2, AF\n2 MP, f/2.4, (depth)
547    16 MP, f/1.7, 1/2.8", 1.12Âµm, PDAF\n2 MP, (de...
548               13 MP, f/2.2, AF\n2 MP, f/2.4, (depth)
549               13 MP, f/2.2, AF\n2 MP, f/2.4, (depth)
550                                                  NaN
Name: Dual, Length: 551, dtype: object

In [5]:
df_raw[['Single', 'Dual', 'Triple']]

,Single,Dual,Triple
0,"8 MP, f/2.0, (wide)",NaN,"64 MP, f/1.8, (wide), PDAF\n5 MP, f/2.2, 115Ëš..."
1,"32 MP, f/2.2, (wide)",NaN,"64 MP, f/1.8, 26mm (wide), PDAF\n12 MP, f/2.2,..."
2,"13 MP, f/2.2, (wide), 1/3.1"", 1.12Âµm",NaN,NaN
3,"13 MP, f/2.2, (wide)",NaN,NaN
4,"5 MP, f/2.2",NaN,"13 MP, f/2.2, (wide), AF\n2 MP, f/2.4, (macro)..."
...,...,...,...
546,"25 MP, f/2.0, 1/2.8"", 0.9Âµm","13 MP, f/2.2, AF\n2 MP, f/2.4, (depth)",NaN
547,"16 MP, f/2.0, 25mm (wide), 1/3.06"", 1.0Âµm","16 MP, f/1.7, 1/2.8"", 1.12Âµm, PDAF\n2 MP, (de...",NaN
548,"5 MP, f/2.2","13 MP, f/2.2, AF\n2 MP, f/2.4, (depth)",NaN
549,"8 MP, f/2.2, 1/4"", 1.12Âµm","13 MP, f/2.2, AF\n2 MP, f/2.4, (depth)",NaN


In [6]:
gpu_ratings = pd.read_excel('../data/gpu_scores.xlsx')

In [7]:
gpu_ratings.head()

,Rank,GPU Name,SOCs,Centurion Mark
0,#1,A14 Bionic’s GPU,Apple A14 Bionic,134.8
1,#2,Adreno 660,Snapdragon 888,131.3
2,#3,A13 Bionic’s GPU,Apple A13 Bionic,126.1
3,#4,Mali-G78 MP24,Kirin 9000,124.1
4,#5,Mali-G78 MP22,Kirin 9000E,122.4


In [9]:
gpu_ratings['Brand'] = gpu_ratings['GPU Name'].replace({'-':' '},regex = True).apply(lambda x: x.split()[0])

In [10]:
gpu_ratings['Brand']= gpu_ratings.Brand.replace('IMG', 'PowerVR')

In [11]:
df_raw.GPU.isnull().sum()

26

In [12]:
df_raw.GPU.astype(str).replace({'-':' '}, regex = True).apply(lambda x: x.split(' ')[0]).value_counts()

Mali       283
Adreno     203
PowerVR     34
nan         26
IMG8322      5
Name: GPU, dtype: int64

In [13]:
df_raw ['Brand_GPU'] = df_raw.GPU.astype(str).replace({'-':' '}, regex = True).apply(lambda x: x.split(' ')[0])

In [14]:
df_raw['Brand_GPU'] = df_raw['Brand_GPU'].replace('IMG8322', 'PowerVR')

In [15]:
match = []
match_score = []
for n,dt in enumerate(df_raw.GPU):
    if dt:
        gpu_matches1 = gpu_ratings[gpu_ratings['Brand'] == df_raw.iloc[n]['Brand_GPU']]
        if len(gpu_matches1) > 0: 
            match_idx = np.argmax([fuzz.ratio(dt, j) for j in gpu_matches1['GPU Name']])
            match.append(gpu_matches1.iloc[match_idx]['GPU Name'])
            match_score.append(gpu_matches1.iloc[match_idx]['Centurion Mark'])
        else:
            
            match.append(None)
            match_score.append(None)
    else:
        print(1)
        match.append(None)
        match_score.append(None)

In [16]:
df_raw['GPU_mapped'] = match

In [17]:
df_raw[['GPU','GPU_mapped']]

,GPU,GPU_mapped
0,Mali-G57 MC2,Mali-G57 MC5
1,Adreno 642L,Adreno 640
2,Mali-G52 MC2,Mali-G52 MC2
3,Mali-G57 MC3,Mali-G57 MC3
4,PowerVR GE8320,PowerVR GE8320
...,...,...
546,Mali-G72 MP3,Mali-G72 MP3
547,Adreno 512,Adreno 512
548,Adreno 506,Adreno 506
549,Adreno 506,Adreno 506


In [18]:
df_raw['GPU_score'] = match_score

In [20]:
df_raw.GPU_score.head()

0    106.2
1    117.7
2     81.6
3     99.4
4     51.3
Name: GPU_score, dtype: float64

In [21]:
cpu_ratings = pd.read_excel('../data/cpu_scores.xlsx')

In [22]:
cpu_ratings

,Rank,Processor Name,Phone,Centurion Mark
0,#1,Apple A14 Bionic,Apple iPhone 12,165
1,#2,Snapdragon 888,OnePlus 9,158
2,#3,Exynos 2100,Samsung Galaxy S21,156
3,#4,Apple A13 Bionic,Apple iPhone 11,154
4,#5,Kirin 9000,Huawei Mate 40 Pro,152
...,...,...,...,...
154,#155,MT6737T,Samsung Galaxy J2 Prime,46
155,#156,Snapdragon 410,Xiaomi Redmi 2,45
156,#157,MT6737,Moto E4 Plus,45
157,#158,MT6735,Meizu M2,42


In [23]:
cpu_ratings['Centurion Mark'][cpu_ratings['Centurion Mark'].astype(str).apply(lambda x: x[-1] == '*')] = cpu_ratings['Centurion Mark'][cpu_ratings['Centurion Mark'].astype(str).apply(lambda x: x[-1] == '*')].apply(lambda x: x[:-1])

In [24]:
cpu_ratings['Centurion Mark'] = cpu_ratings['Centurion Mark'].astype(str)

In [25]:
df_raw.Chipset

0        MediaTek MT6833 Dimensity 700 5G (7 nm)
1      Qualcomm SM7325 Snapdragon 778G 5G (6 nm)
2                     Mediatek Helio G80 (12 nm)
3        MediaTek MT6853 Dimensity 720 5G (7 nm)
4               MediaTek MT6765 Helio P35 (12nm)
                         ...                    
546             Mediatek MT6771 Helio P70 (12nm)
547       Qualcomm SDM660 Snapdragon 660 (14 nm)
548       Qualcomm SDM450 Snapdragon 450 (14 nm)
549       Qualcomm SDM450 Snapdragon 450 (14 nm)
550            Mediatek MT6771 Helio P60 (12 nm)
Name: Chipset, Length: 551, dtype: object

In [26]:
df_raw['Brand_CPU'] = df_raw.Chipset.astype(str).replace({'-':' '}, regex = True).apply(lambda x: x.split(' ')[0].lower())

In [27]:
df_raw['Brand_CPU'] = df_raw['Brand_CPU'].replace({'qualcomm':'snapdragon'})

In [29]:
cpu_ratings['Brand'] = cpu_ratings['Processor Name'].replace({'-':' '},regex = True).apply(lambda x: x.split()[0].lower())

In [30]:
cpu_ratings['Brand'] = cpu_ratings['Brand'].replace({'qualcomm':'snapdragon'})


In [31]:
cpu_ratings['Brand'][cpu_ratings['Brand'].apply(lambda x: x[:2] == 'mt')] = 'mediatek'

In [32]:
cpu_ratings.Brand.value_counts()

snapdragon    55
helio         30
exynos        25
kirin         20
dimensity     11
mediatek      10
apple          8
Name: Brand, dtype: int64

In [33]:
cpu_ratings['Centurion Mark']

0      165
1      158
2      156
3      154
4      152
      ... 
154     46
155     45
156     45
157     42
158     41
Name: Centurion Mark, Length: 159, dtype: object

In [35]:
match = []
match_score = []
for n,dt in enumerate(df_raw.Chipset):
    if dt:
        cpu_matches1 = cpu_ratings[cpu_ratings['Brand'] == df_raw.iloc[n]['Brand_CPU']]
        if len(cpu_matches1) > 0: 
            match_idx = np.argmax([fuzz.ratio(dt, j) for j in cpu_matches1['Processor Name']])
            match.append(cpu_matches1.iloc[match_idx]['Processor Name'])
            match_score.append(cpu_matches1.iloc[match_idx]['Centurion Mark'])
        else:
            
            match.append(None)
            match_score.append(None)
    else:
        print(1)
        match.append(None)
        match_score.append(None)

In [36]:
df_raw['CPU_mapped'] = match

In [37]:
df_raw['CPU_score'] = match_score

In [38]:
df_raw.CPU_score.isnull().sum()

36

In [39]:
df_raw[['Chipset','CPU_mapped']]

,Chipset,CPU_mapped
0,MediaTek MT6833 Dimensity 700 5G (7 nm),MT6750T
1,Qualcomm SM7325 Snapdragon 778G 5G (6 nm),Snapdragon 780G
2,Mediatek Helio G80 (12 nm),MT6750T
3,MediaTek MT6853 Dimensity 720 5G (7 nm),MT6750T
4,MediaTek MT6765 Helio P35 (12nm),MT6753
...,...,...
546,Mediatek MT6771 Helio P70 (12nm),MT6750T
547,Qualcomm SDM660 Snapdragon 660 (14 nm),Snapdragon 660
548,Qualcomm SDM450 Snapdragon 450 (14 nm),Snapdragon 450
549,Qualcomm SDM450 Snapdragon 450 (14 nm),Snapdragon 450


In [111]:
df_raw['Brand_CPU']

0        mediatek
1      snapdragon
2        mediatek
3        mediatek
4        mediatek
5      snapdragon
6      snapdragon
7      snapdragon
8          exynos
9          exynos
10         exynos
11         exynos
12         exynos
13       mediatek
14       mediatek
15       mediatek
16       mediatek
17       mediatek
18     snapdragon
19     snapdragon
20     snapdragon
21         exynos
22     snapdragon
23         exynos
24     snapdragon
25     snapdragon
26     snapdragon
27     snapdragon
28         exynos
29       mediatek
          ...    
521    snapdragon
522      mediatek
523    snapdragon
524      mediatek
525    snapdragon
526      mediatek
527      mediatek
528    snapdragon
529    snapdragon
530    snapdragon
531      mediatek
532      mediatek
533    snapdragon
534    snapdragon
535    snapdragon
536    snapdragon
537    snapdragon
538    snapdragon
539    snapdragon
540      mediatek
541    snapdragon
542    snapdragon
543      mediatek
544      mediatek
545    sna

In [40]:
match

['MT6750T',
 'Snapdragon 780G',
 'MT6750T',
 'MT6750T',
 'MT6753',
 'Snapdragon 780G',
 'Snapdragon 888',
 'Snapdragon 888',
 'Exynos 9825',
 'Exynos 9825',
 'Exynos 850',
 'Exynos 850',
 'Exynos 9611',
 'MT6750T',
 'MT6750T',
 'MT6750T',
 'MT6750T',
 'MT6752',
 'Snapdragon 750G',
 'Snapdragon 750G',
 'Snapdragon 750G',
 'Exynos 850',
 'Snapdragon 855+',
 'Exynos 850',
 'Snapdragon 450',
 'Snapdragon 720G',
 'Snapdragon 750G',
 'Snapdragon 720G',
 'Exynos 850',
 'MT6750T',
 'Exynos 9825',
 'Exynos 9825',
 'Exynos 850',
 'Exynos 2100',
 'Exynos 2100',
 'Exynos 2100',
 'MT6750T',
 'Snapdragon 450',
 'MT6753',
 'MT6739',
 'MT6739',
 'Snapdragon 450',
 'Exynos 9611',
 'Exynos 9611',
 'Exynos 9611',
 'Exynos 9810',
 'Snapdragon 865+',
 'Exynos 9825',
 'Snapdragon 750G',
 'Snapdragon 662',
 'Snapdragon 730G',
 'Snapdragon 765G',
 'Snapdragon 865+',
 'Exynos 9825',
 'Exynos 9825',
 'Exynos 9825',
 'Exynos 9825',
 'Exynos 2100',
 'Snapdragon 865+',
 'Snapdragon 865+',
 'Snapdragon 865+',
 'Exy

In [41]:
df_raw[['name','CPU_mapped','CPU_score', 'GPU_mapped','GPU_score']].to_clipboard()

In [138]:
# df_processed = pd.read_clipboard()

In [139]:
(df_processed.notnull().sum()/551).sort_values()

Screen to body ratio    0.762250
Wattage                 0.776770
F                       0.805808
Single-MP               0.843920
CPU_score               0.931034
CPU_mapped              0.931034
nanometers              0.934664
GPU_mapped              0.949183
GPU_score               0.949183
Price                   0.976407
Ram                     0.978221
Internal                0.983666
Thickness               0.987296
Announced               0.990926
Weight                  0.994555
PPI                     0.994555
Size                    0.996370
Wireless Charging       0.996370
name                    0.996370
#2g_bands               0.996370
#3g_bands               0.996370
#4g_bands               0.996370
#5g_bands               0.996370
Bluetooth               0.996370
VERSION                 0.996370
OS                      0.996370
Battery                 0.996370
Brand                   0.996370
NFC                     0.996370
#cameras                0.996370
Total_band

In [130]:
df_processed['Screen to body ratio'].describe()

count    420.000000
mean      80.049048
std        6.939821
min       56.000000
25%       75.275000
50%       82.200000
75%       84.200000
max       95.900000
Name: Screen to body ratio, dtype: float64